In [1]:
# 所有的头

In [2]:
import torch # for save
import os
from van import Fan
from localconfig import config
import datetime
import random
from pprint import pprint
import html
from pyltp import SentenceSplitter
from pyltp import Postagger
from pyltp import Segmentor
from pyltp import NamedEntityRecognizer
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import Image
from IPython.core.display import HTML 
import pyecharts
from pyecharts import Pie
from pyecharts import Line

%matplotlib inline

plt.rcParams['font.sans-serif']=['FangSong'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号


In [3]:
# 所有的配置

In [4]:
keyword = '美甲' # 关注的关键词
until_year = 2000 # 最早的年份
until_num = 5000 # 截止的饭否数量
timeout = (60,60) # 超时时间

In [5]:
# 登录
fan = Fan(config['Consumer key'], config['Consumer secret'])
fan.xauth(config['my username'], config['my password'])

{'oauth_token': '1405515-071ad3666b5af9441d5cbb0dc1d3985b',
 'oauth_token_secret': '43cc293f5d127423f960b8636688577b'}

In [6]:
# 本小时内可用API次数
fan.request('GET', 'account/rate_limit_status', {}, timeout=timeout)

{'reset_time': 'Mon Mar 18 09:16:05 +0000 2019',
 'remaining_hits': 150,
 'hourly_limit': 150,
 'reset_time_in_seconds': 1552900565}

In [7]:
# 采集关键词消息
def datetimefromstr(s):
    return datetime.datetime.strptime(s, '%a %b %d %H:%M:%S %z %Y')
def get_status(keyword, until_year, until_num):
    latest = fan.request('GET', 'search/public_timeline', {'q':keyword, 'count':60}, timeout=timeout)
    
    while True:
        year = datetimefromstr(latest[-1]['created_at']).year
        if len(latest) >= until_num:
            break
        if year < until_year:
            break
        print(len(latest), year,'/', until_num, until_year)
        len_ = len(latest)
        latest += fan.request('GET', 'search/public_timeline', { 'q':keyword, 'count':60, 'max_id':latest[-1]['id']}, timeout=timeout)[1:]
        if len(latest) == len_:
            break
    return latest

In [8]:
# 下载并暂存
statuses = get_status(keyword, until_year, until_num)

60 2019 / 5000 2000
119 2019 / 5000 2000
178 2018 / 5000 2000
237 2018 / 5000 2000
296 2018 / 5000 2000
355 2018 / 5000 2000
414 2018 / 5000 2000
473 2018 / 5000 2000
532 2018 / 5000 2000
591 2017 / 5000 2000
650 2017 / 5000 2000
709 2017 / 5000 2000
768 2017 / 5000 2000
827 2017 / 5000 2000
886 2017 / 5000 2000
945 2016 / 5000 2000
1004 2016 / 5000 2000
1063 2016 / 5000 2000
1122 2015 / 5000 2000
1181 2015 / 5000 2000
1240 2015 / 5000 2000
1299 2015 / 5000 2000
1358 2014 / 5000 2000
1417 2014 / 5000 2000
1476 2014 / 5000 2000
1535 2013 / 5000 2000
1594 2013 / 5000 2000
1653 2013 / 5000 2000
1712 2013 / 5000 2000
1771 2013 / 5000 2000
1830 2013 / 5000 2000
1889 2013 / 5000 2000
1948 2013 / 5000 2000
2007 2013 / 5000 2000
2066 2013 / 5000 2000
2125 2013 / 5000 2000
2184 2013 / 5000 2000
2243 2013 / 5000 2000
2302 2013 / 5000 2000
2361 2013 / 5000 2000
2420 2013 / 5000 2000
2479 2013 / 5000 2000
2538 2012 / 5000 2000
2597 2012 / 5000 2000
2656 2012 / 5000 2000
2715 2012 / 5000 2000
2774 

In [9]:
torch.save(statuses,'tmp_statuses.torch.pkl')

In [ ]:
# 恢复
# statuses = torch.load('tmp_statuses.torch.pkl')

In [10]:
print(len(statuses))

5016


In [11]:
# common
def get_top(l, num=5, reverse=False): # 从大到小
    cnt = {}
    for x in l:
        if type(x) == tuple and len(x) == 3:
            x = x[0]
        if x not in cnt:
            cnt[x] = 0
        cnt[x] += 1
    cnt_list = [(k,cnt[k],cnt[k]/len(l)) for k in cnt]
    cnt_list = sorted(cnt_list, key=lambda x:x[1])[::-1]
    if reverse:
        cnt_list = cnt_list[::-1]
    if num == -1:
        return cnt_list
    return cnt_list[:num]
def huabing(y): # x [(label,,0.xxx)]
    attr = [x[0] for x in y]
    v1 = [x[2] for x in y]
    if sum(v1) < 1.0:
        attr.append('其他')
        v1.append(1.0 - sum(v1))
    pie = Pie('')
    pie.add('', attr, v1, is_label_show=True)
    return pie

In [12]:
# 从消息关系本身入手

In [13]:
# 时间段分布

In [14]:
print(datetimefromstr(statuses[-1]['created_at']))
print(datetimefromstr(statuses[0]['created_at']))
begin_year = datetimefromstr(statuses[-1]['created_at']).year
end_year = datetimefromstr(statuses[0]['created_at']).year

2011-08-15 12:46:26+00:00
2019-03-16 14:44:31+00:00


In [15]:
huabing(get_top([datetimefromstr(x['created_at']).year for x in statuses], num = -1))

In [16]:
arr = sorted(get_top([datetimefromstr(x['created_at']).year for x in statuses],num=-1),key = lambda x:x[0])
attr = [str(x[0]) for x in arr ]
v1 = [x[1] for x in arr ]
line = Line(keyword)
line.add('', attr, v1,is_smooth=True,is_label_show=True,mark_line=[ "average"])
# line.add("商家B", attr, v2, is_smooth=True, mark_line=["max", "average"])
line

In [17]:
# 用户信息文件
os.path.exists('./users.torch.pkl')

True

In [18]:
# 预处理用户信息
userid_to_info = {}
if os.path.exists('./users.torch.pkl'):
    userid_to_info = torch.load('users.torch.pkl')
for x in statuses:
    userid_to_info[x['user']['id']] = x['user']
torch.save(userid_to_info, 'users.torch.pkl')
print(len(userid_to_info))

5461


In [19]:
# 发表人数/总消息数量
uni_num = len(set([x['user']['id'] for x in statuses ]))
print(uni_num, len(statuses), uni_num/len(statuses))

2018 5016 0.40231259968102073


In [55]:
# 发表top30的人
users_top = get_top([x['user']['id'] for x in statuses], num=30)
names_top = [ (userid_to_info[x[0]]['name'] , x[0], x[1], x[2]) for x in users_top]
pprint(names_top)
huabing(names_top)

[('米粒煮妇', '~I7gKMEORlDo', 133, 0.026515151515151516),
 ('诗意地栖居', 'chenjl8160', 95, 0.01893939393939394),
 ('超人小九', 'soul_nine', 91, 0.01814194577352472),
 ('真人美甲秀', 'xiaoqi2005', 71, 0.014154704944178628),
 ('lymi沐沐', '~vaDEK4rh7jA', 70, 0.013955342902711323),
 ('momoko1031', '~k7iIkK6ax9Y', 62, 0.012360446570972886),
 ('糗百', 'qiub', 56, 0.011164274322169059),
 ('幻语星辰', '~LIVgLdcX5YE', 51, 0.010167464114832535),
 ('国色天香美甲', 'gstxmj', 40, 0.007974481658692184),
 ('anne312', '~GkH_gx0TeY8', 37, 0.007376395534290271),
 ('cici', 'cicisj', 37, 0.007376395534290271),
 ('透透_颖', '~7y5AKnMT7ao', 36, 0.007177033492822967),
 ('林雯倩', '~Er7FZUBRu_g', 31, 0.006180223285486443),
 ('爱上美', 'aishmeizp', 28, 0.005582137161084529),
 ('開開頭', '~X5mTdjCtYz0', 26, 0.00518341307814992),
 ('A梦迪迪', '~ICObstPqU24', 23, 0.004585326953748007),
 ('2kkk', 'callcrazy', 22, 0.0043859649122807015),
 ('Joly.ph1202', '~s5DY32QVdds', 21, 0.004186602870813397),
 ('codimy', 'codimy', 21, 0.004186602870813397),
 ('时尚瑞丽美甲', 'r

In [21]:
def show_user_info(x):
    user = userid_to_info[x]
    pprint(user)
    return Image(url = user['profile_image_url_large'])

In [22]:
show_user_info(users_top[0][0])

{'birthday': '1986-02-15',
 'created_at': 'Tue Feb 21 06:38:04 +0000 2012',
 'description': '',
 'favourites_count': 0,
 'followers_count': 3,
 'following': False,
 'friends_count': 0,
 'gender': '女',
 'id': '~I7gKMEORlDo',
 'location': '上海 杨浦区',
 'name': '米粒煮妇',
 'notifications': False,
 'photo_count': 6231,
 'profile_background_color': '#acdae5',
 'profile_background_image_url': 'http://static.fanfou.com/img/bg/0.png',
 'profile_background_tile': False,
 'profile_image_url': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/s0/00/00/00.jpg',
 'profile_image_url_large': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/l0/00/00/00.jpg',
 'profile_link_color': '#0066cc',
 'profile_sidebar_border_color': '#b2d1a3',
 'profile_sidebar_fill_color': '#e2f2da',
 'profile_text_color': '#222222',
 'protected': False,
 'screen_name': '米粒煮妇',
 'statuses_count': 7309,
 'unique_id': '~I7gKMEORlDo',
 'url': '',
 'utc_offset': 28800}


In [56]:
# 被引用top20的消息

repost_top = get_top([x['repost_status']['id'] for x in statuses if 'repost_status' in x], num=20)
for i in range(len(repost_top)):
    x = repost_top[i]
    for y in statuses:
        if 'repost_status' in y and y['repost_status']['id'] == x[0]:
            repost_top[i] = ('id: '+y['repost_status']['user']['id'] + ' name: '+y['repost_status']['user']['name']  +' '+ y['repost_status']['text'],x[0], x[1],x[2])
            break
            
pprint(repost_top)

[('id: summeronion name: 令糊葱 '
  '“我对凤姐缺乏了解，但听同事说，凤姐微博上四百八十万粉丝，如果一月接一次广告，就不用在美国做美甲，可她的微博从来不接广告，这一点，范冰冰都做不到。”',
  '5E4Vt7hy6RU',
  37,
  0.11455108359133127),
 ('id: ~7HMFE9-mkN8 name: 烧烧烧烧烤 美甲已做好，可是面基的人还未出现，烧烤只能撤了^_^',
  'LN8VhyDlYB4',
  9,
  0.02786377708978328),
 ('id: ningqun name: 一介大叔 '
  '在海底捞的美甲区坐上半小时，就能从美甲师口中获得浓缩了人间况味的员工传奇。比如一位富婆离婚后来这里当服务员疗伤，拿着三四千的工资每月消费上万，端茶送水毫不怠慢；一个女老板在海底捞发了一个月热毛巾，后来开了家颇有名气的火锅店。',
  'Sl7VPAjhtu0',
  8,
  0.02476780185758514),
 ('id: maggiepan name: 纯圆鹅 我朋友圈的美甲师发的哈哈哈哈哈哈哈哈哈哈',
  'o2z4-S6osEk',
  8,
  0.02476780185758514),
 ('id: velvet1981 name: 地下天鹅绒 '
  '北京地铁票价涨这么多，涨价可以，服务完全跟不上啊，像十三号线五道口这种晚高峰排队能排到街上半里地，冻一刻钟还得挤成狗才能进去的丧病地铁站，是不是得给等车的乘客赠送瓜子、炸虾片、热豆浆，还有提供免费美甲？要不北京公共交通怎么和国际接轨，跟海底捞都接不上轨。',
  'FXm0-jeAiA0',
  6,
  0.018575851393188854),
 ('id: nyaball name: S.N.Nyaball 一辈子写一个系列',
  '4fc6FIFWxdw',
  6,
  0.018575851393188854),
 ('id: ~q3PAKFOIRO8 name: 果照 指甲的作用都有什么？抠鼻子？挠痒痒？挖耳屎？挤青春痘？所以，女孩为什么都要美甲？？？',
  '0N2iCGJvMpg',
  5,
  0.015479876160990712),
 ('id: lovejury name: 阿

In [57]:
# 引用top20的人
repost_user_id_top = get_top([x['repost_status']['user']['id'] for x in statuses if 'repost_status' in x], num=20)
repost_name_top =  [ (userid_to_info[x[0]]['name'] , x[0], x[1], x[2]) for x in repost_user_id_top]
pprint(repost_name_top)

[('令糊葱', 'summeronion', 37, 0.11455108359133127),
 ('烧烧烧烧烤', '~7HMFE9-mkN8', 11, 0.034055727554179564),
 ('果照', '~q3PAKFOIRO8', 9, 0.02786377708978328),
 ('Push__', 'olivia0919', 9, 0.02786377708978328),
 ('青瓦的旧时光', 'blessedkristin', 9, 0.02786377708978328),
 ('一介大叔', 'ningqun', 8, 0.02476780185758514),
 ('多晴.', 'hanxuuu', 8, 0.02476780185758514),
 ('纯圆鹅', 'maggiepan', 8, 0.02476780185758514),
 ('地下天鹅绒', 'velvet1981', 7, 0.021671826625386997),
 ('轻轻河边操', 'nicetomeetyou', 6, 0.018575851393188854),
 ('S.N.Nyaball', 'nyaball', 6, 0.018575851393188854),
 ('糗百', 'qiub', 5, 0.015479876160990712),
 ('徒有实名', '~DMkLPYc13EM', 5, 0.015479876160990712),
 ('左堂堂', 'bb0813', 4, 0.01238390092879257),
 ('2kkk', 'callcrazy', 4, 0.01238390092879257),
 ('阿拉瓜丝逼', 'lovejury', 4, 0.01238390092879257),
 ('RrcanR', 'wordsofpipa', 4, 0.01238390092879257),
 ('二他姐姐', 'znz28', 4, 0.01238390092879257),
 ('eeleven', '~_zH1Qn7pnFs', 4, 0.01238390092879257),
 ('寒小梦', 'cincylan', 3, 0.009287925696594427)]


In [25]:
# top20 gender
gender_top = get_top([x['user']['gender'] for x in statuses ], num=20)
pprint(gender_top)
huabing(gender_top)

[('', 2224, 0.4433811802232855),
 ('女', 2110, 0.4206539074960128),
 ('男', 682, 0.13596491228070176)]


In [26]:
# top10 location
location_top10 = get_top
location_top10 = get_top([x['location'].split()[0] for x in statuses if len(x['location'].split())>=1], num=10)
pprint(location_top10)
huabing(location_top10)

[('北京', 294, 0.07040229885057471),
 ('上海', 219, 0.05244252873563218),
 ('广东', 170, 0.040708812260536395),
 ('39.840050,116.447290', 128, 0.03065134099616858),
 ('31.236601,121.484596', 123, 0.029454022988505746),
 ('江苏', 105, 0.02514367816091954),
 ('浙江', 100, 0.023946360153256706),
 ('31.272423,121.477548', 68, 0.016283524904214558),
 ('31.193535,121.514226', 60, 0.014367816091954023),
 ('山东', 60, 0.014367816091954023)]


In [27]:
def remove_atag(s):
    if '<a' not in s:
        return s
    start = s.find('>')
    end = s.find('</a>',start)
    return s[start+1:end]

In [28]:
# top20 source
source_top = get_top([remove_atag(x['source']) for x in statuses ], num=20)
pprint(source_top)
huabing(source_top)

[('街旁', 2446, 0.4876395534290271),
 ('网页', 546, 0.10885167464114832),
 ('御飯 iOS', 263, 0.052432216905901115),
 ('「御飯 iPhone」', 257, 0.05123604465709729),
 ('Android客户端', 199, 0.03967304625199362),
 ('享拍微博通', 181, 0.036084529505582136),
 ('猫饭', 175, 0.03488835725677831),
 ('开开Kai', 126, 0.025119617224880382),
 ('饭否分享', 125, 0.02492025518341308),
 ('手机上网', 116, 0.023125996810207338),
 ('蘑菇饭', 59, 0.011762360446570972),
 ('糗事统计', 52, 0.01036682615629984),
 ('尚饭', 47, 0.009370015948963317),
 ('泡饭', 42, 0.008373205741626795),
 ('API', 41, 0.00817384370015949),
 ('iPhone客户端', 36, 0.007177033492822967),
 ('FaWave', 23, 0.004585326953748007),
 ('饭团fantuan', 22, 0.0043859649122807015),
 ('社交媒体连接', 20, 0.003987240829346092),
 ('有饭', 20, 0.003987240829346092)]


In [29]:
# nlp part

In [30]:
def myfilter(s):
    s = s.replace('<strong>','')
    s = s.replace('</strong>','')
    while '转@' in s:
        pos = s.find('转@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@']:
            end2 = s.find(endsym,pos+2)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [转@] too long',s)
            s = s[:pos] + s[pos+2:]
        else:
            s = s[:pos] + s[end:]
    while '@' in s:
        pos = s.find('@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@']:
            end2 = s.find(endsym,pos+1)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [@] too long',s)
            s = s[:pos] + s[pos+1:]
        else:
            s = s[:pos] + s[end:]
    if '@' in s:
        print('alert @ in ', s)
    return s

In [31]:
sentences = []
for x in tqdm(statuses):
    s = html.unescape(x['text']) 
    s = myfilter(s)
    if keyword not in s:
        continue
    if '</' in s:
        print(s)
    ss = SentenceSplitter.split(s)
    for t in ss:
        if len(t) > 4:
            sentences.append(t)
    

100%|██████████| 5016/5016 [00:00<00:00, 22607.11it/s]

alert [@] too long http://to.taobao.com/SnsQ4ky每天都会不定时更新真人美甲秀的哦~有好看的美甲秀可以@我，大家一起秀出自己的真人秀哦~指甲油赛高！最爱OPI~
alert [@] too long 我在Lily Nails：来个素雅的。@三里屯lilynails美甲造型 http://jie.pn/p_Ic0W9AAF
alert [@] too long 我在[活动] 三里屯Village - CONVERSE新店开幕：亲爱的娜娜@Summer驴儿，时隔八年在相聚，杂就那短暂？你好好的，从日本回来咱在聚，这次不能陪你东京各种耍了，我在北京等你哈美甲ing http://jie.pn/p_Ic1E1xED
alert [@] too long 我在夏花美甲(魏公村店)：亲爱的娜娜，@Summer驴儿，我已经开始了！ http://jie.pn/p_Ic0PzxED
alert [@] too long 我在世博公园(Expo Park)：等亲爱的娜娜小姐@Summer驴儿，来接我去美甲。遛狗巧遇去南极搞科普的狗西西，尼玛穿这么多，主人脑子有包吧！ http://jie.pn/p_Ic2zfBED
alert [@] too long 同感哈哈哈哈 顶…  腾讯微博无故@我的人，我全给他们举报了！md！真烦！每天都是化妆美甲，有完没完！
alert [@] too long 我在長春國賓：(@)听美甲小姑娘说她朋友生了对双胞胎男孩，真让人兴奋~外人听了都觉得开心（＾∇＾） http://jie.pn/s4z2e1h
alert [@] too long 我在Nana时尚美甲沙龙：陪@徐美美_yUE换颜色，动心忍性动心忍性~ http://jie.pn/s6z6m2c


In [32]:
print(len(sentences))

9678


In [33]:
# 对sentences去重。。 拒绝转发的影响 （存疑？）
# 不去重

In [34]:
# sentences = list(set(sentences))

In [35]:
print(len(list(set(sentences))))

8061


In [36]:
# torch.save(['尼玛', '卧槽', '傻逼', '对了', '有木有', '他妈'],'blacklist.torch.pkl' )
def add_to_blacklist(w):
    global blacklist
    blacklist = torch.load('blacklist.torch.pkl')
    blacklist.append(w)
    torch.save(blacklist,'blacklist.torch.pkl')

In [37]:
# 无意义
blacklist = torch.load('blacklist.torch.pkl')

In [38]:
words_all = []

segmentor = Segmentor()
postagger = Postagger()
recognizer = NamedEntityRecognizer()
postagger.load('./ltp_data_v3.4.0/pos.model')
segmentor.load('./ltp_data_v3.4.0/cws.model')
recognizer.load('./ltp_data_v3.4.0/ner.model')

#todo 情感分析

for s in tqdm(sentences):
    words = segmentor.segment(s)
    postags = postagger.postag(words) #词性标注
    netags = recognizer.recognize(words, postags) # 命名实体识别
    for i in range(len(words)):
        if words[i] in blacklist:
            continue
        words_all.append((words[i],postags[i],netags[i]))

segmentor.release()
postagger.release()
recognizer.release()

100%|██████████| 9678/9678 [00:09<00:00, 1003.73it/s]


In [39]:
print(len(words_all))

131844


In [40]:
get_top([x[0] for x in words_all if len(x[0])>1 and x[1]!='wp'], num = 25)

[('美甲', 3517, 0.06350324106675334),
 ('指甲', 710, 0.012819818355813155),
 ('美甲店', 551, 0.00994890128739866),
 ('一个', 410, 0.007402993698427315),
 ('化妆', 254, 0.004586244876586678),
 ('学校', 248, 0.004477908383438961),
 ('可以', 238, 0.0042973475615261),
 ('自己', 234, 0.004225123232760955),
 ('美容', 229, 0.004134842821804525),
 ('今天', 203, 0.0036653846848310853),
 ('培训', 194, 0.00350287994510951),
 ('分享', 181, 0.0032681508766227905),
 ('什么', 180, 0.0032500947944315043),
 ('Nail', 179, 0.003232038712240218),
 ('觉得', 176, 0.00317787046566636),
 ('沙龙', 159, 0.0028709170684144956),
 ('美甲师', 159, 0.0028709170684144956),
 ('昆明', 158, 0.0028528609862232095),
 ('指甲油', 158, 0.0028528609862232095),
 ('现在', 153, 0.002762580575266779),
 ('凤姐', 150, 0.0027084123286929204),
 ('朋友', 145, 0.0026181319177364893),
 ('美甲图', 134, 0.002419515013632342),
 ('喜欢', 134, 0.002419515013632342),
 ('专业', 133, 0.0024014589314410558)]

In [41]:
posdict = {}
nedict = {}
for x in tqdm(words_all):
    if x[1] not in posdict:
        posdict[x[1]] = []
    if x[2] not in nedict:
        nedict[x[2]] = []
    posdict[x[1]].append(x)
    nedict[x[2]].append(x)

100%|██████████| 131844/131844 [00:00<00:00, 821029.60it/s]


In [42]:
# 词性
print(posdict.keys())


dict_keys(['a', 'n', 'v', 'wp', 'r', 'u', 'm', 'q', 'nz', 'nt', 'd', 'ws', 'c', 'o', 'j', 'nd', 'p', 'nl', 'nh', 'i', 'b', 'ns', 'e', 'z', 'k', '%', 'ni'])


In [43]:
# ['i'] 成语
pprint(get_top(posdict['i'], num=25))

[('甲天下', 26, 0.03951367781155015),
 ('爱不释手', 21, 0.031914893617021274),
 ('差不多', 14, 0.02127659574468085),
 ('十方淘', 10, 0.015197568389057751),
 ('千艺美甲', 8, 0.0121580547112462),
 ('美甲', 8, 0.0121580547112462),
 ('伤不起', 8, 0.0121580547112462),
 ('过日子', 7, 0.010638297872340425),
 ('捞面舞', 6, 0.00911854103343465),
 ('各种各样', 6, 0.00911854103343465),
 ('独树一帜', 5, 0.007598784194528876),
 ('有意思', 5, 0.007598784194528876),
 ('说不定', 5, 0.007598784194528876),
 ('喜怒哀乐', 5, 0.007598784194528876),
 ('无所谓', 5, 0.007598784194528876),
 ('是的', 5, 0.007598784194528876),
 ('没什么', 5, 0.007598784194528876),
 ('而立之年', 5, 0.007598784194528876),
 ('不怎么', 5, 0.007598784194528876),
 ('异口同声', 4, 0.0060790273556231),
 ('莫名其妙', 4, 0.0060790273556231),
 ('不得了', 4, 0.0060790273556231),
 ('白富美', 4, 0.0060790273556231),
 ('心旷神怡', 3, 0.004559270516717325),
 ('相映成趣', 3, 0.004559270516717325)]


In [44]:

# ['j'] 缩略词
pprint(get_top(posdict['j'], num=25))


[('美', 159, 0.28191489361702127),
 ('美甲', 68, 0.12056737588652482),
 ('埃洛美甲', 42, 0.07446808510638298),
 ('微博', 38, 0.0673758865248227),
 ('芊芊', 16, 0.028368794326241134),
 ('夏花美甲', 10, 0.01773049645390071),
 ('日', 10, 0.01773049645390071),
 ('小企', 8, 0.014184397163120567),
 ('尚彩·美甲', 7, 0.012411347517730497),
 ('苏', 7, 0.012411347517730497),
 ('哈', 7, 0.012411347517730497),
 ('美美', 7, 0.012411347517730497),
 ('欧卡奴', 6, 0.010638297872340425),
 ('伊', 6, 0.010638297872340425),
 ('高姐', 6, 0.010638297872340425),
 ('吉恩美甲', 5, 0.008865248226950355),
 ('呆呆美甲', 5, 0.008865248226950355),
 ('沪', 5, 0.008865248226950355),
 ('阿', 5, 0.008865248226950355),
 ('三玉指', 4, 0.0070921985815602835),
 ('玉指', 4, 0.0070921985815602835),
 ('青青', 4, 0.0070921985815602835),
 ('美妆', 4, 0.0070921985815602835),
 ('女足', 4, 0.0070921985815602835),
 ('韩', 4, 0.0070921985815602835)]


In [45]:

# ['n'] 名词
pprint(get_top(posdict['n'], num=25))


[('美甲', 2966, 0.12006638869772902),
 ('指甲', 689, 0.027891349228838604),
 ('美甲店', 549, 0.022224021373922195),
 ('店', 549, 0.022224021373922195),
 ('人', 353, 0.014289762377039225),
 ('学校', 248, 0.010039266485851921),
 ('手', 204, 0.008258106302878192),
 ('甲', 199, 0.008055701736631178),
 ('沙龙', 159, 0.006436465206655062),
 ('美甲师', 159, 0.006436465206655062),
 ('指甲油', 146, 0.005910213334412824),
 ('朋友', 145, 0.005869732421163422),
 ('美甲图', 134, 0.005424442375419989),
 ('颜色', 131, 0.005302999635671781),
 ('专业', 129, 0.005222037809172975),
 ('时候', 123, 0.0049791523296765575),
 ('时尚', 121, 0.004898190503177751),
 ('造型', 120, 0.004857709589928349),
 ('钱', 108, 0.004371938630935514),
 ('同事', 101, 0.004088572238189693),
 ('美妆', 99, 0.004007610411690888),
 ('时间', 93, 0.00376472493219447),
 ('广告', 87, 0.003521839452698053),
 ('照片', 87, 0.003521839452698053),
 ('广场', 82, 0.0033194348864510385)]


In [46]:

# ['nh'] 人名词
pprint(get_top(posdict['nh'], num=25))


[('凤姐', 150, 0.11286681715575621),
 ('昕薇', 92, 0.0692249811888638),
 ('林尚美甲', 67, 0.05041384499623777),
 ('刘娟', 59, 0.04439428141459744),
 ('埃洛美甲', 55, 0.04138449962377728),
 ('芭比', 41, 0.030850263355906696),
 ('冰冰', 38, 0.028592927012791574),
 ('范', 38, 0.028592927012791574),
 ('安娜', 33, 0.024830699774266364),
 ('周', 21, 0.01580135440180587),
 ('刘娟美', 17, 0.012791572610985704),
 ('阿叔', 13, 0.009781790820165538),
 ('萍萍', 11, 0.008276899924755455),
 ('张霞', 10, 0.007524454477050414),
 ('尚', 9, 0.006772009029345372),
 ('琳琳', 9, 0.006772009029345372),
 ('芳芳', 8, 0.006019563581640331),
 ('💅', 8, 0.006019563581640331),
 ('林尚', 8, 0.006019563581640331),
 ('陈', 8, 0.006019563581640331),
 ('蔡', 8, 0.006019563581640331),
 ('罗玉凤', 7, 0.005267118133935289),
 ('小惠', 7, 0.005267118133935289),
 ('依林', 7, 0.005267118133935289),
 ('陆惠', 6, 0.004514672686230248)]


In [47]:

# ['ni'] 组织名
pprint(get_top(posdict['ni'], num=25))


[('美甲8', 1, 0.3333333333333333),
 ('新华社', 1, 0.3333333333333333),
 ('腾讯', 1, 0.3333333333333333)]


In [48]:

# ['nz'] other proper name
pprint(get_top(posdict['nz'], num=25))


[('美甲', 22, 0.0738255033557047),
 ('海底捞', 17, 0.05704697986577181),
 ('中文', 13, 0.0436241610738255),
 ('瑞丽', 12, 0.040268456375838924),
 ('淘宝', 12, 0.040268456375838924),
 ('灵灵', 10, 0.03355704697986577),
 ('雪莉', 7, 0.02348993288590604),
 ('东华', 6, 0.020134228187919462),
 ('新世界', 5, 0.016778523489932886),
 ('中华', 5, 0.016778523489932886),
 ('丝柏娜', 5, 0.016778523489932886),
 ('前卫', 4, 0.013422818791946308),
 ('日文', 4, 0.013422818791946308),
 ('置地', 4, 0.013422818791946308),
 ('大宝', 4, 0.013422818791946308),
 ('星巴克', 4, 0.013422818791946308),
 ('三星', 4, 0.013422818791946308),
 ('英文', 4, 0.013422818791946308),
 ('家乐福', 3, 0.010067114093959731),
 ('诺基亚', 3, 0.010067114093959731),
 ('港汇', 3, 0.010067114093959731),
 ('日语', 3, 0.010067114093959731),
 ('雅诗兰黛', 3, 0.010067114093959731),
 ('常春藤', 2, 0.006711409395973154),
 ('中凯', 2, 0.006711409395973154)]


In [49]:

# ['v'] verb len >= 2
pprint(get_top([x for x in posdict['v'] if len(x[0])>=2], num=25))


[('美甲', 449, 0.0383073116628274),
 ('化妆', 254, 0.021670505929528197),
 ('可以', 238, 0.020305434689872878),
 ('美容', 210, 0.01791656002047607),
 ('培训', 194, 0.016551488780820748),
 ('分享', 181, 0.015442368398600801),
 ('觉得', 176, 0.015015783636208514),
 ('喜欢', 134, 0.0114324716321133),
 ('摄影', 110, 0.009384864772630322),
 ('知道', 100, 0.008531695247845746),
 ('服务', 95, 0.008105110485453459),
 ('看到', 86, 0.007337257913147342),
 ('开始', 85, 0.007251940960668885),
 ('还有', 83, 0.00708130705571197),
 ('工作', 79, 0.00674003924579814),
 ('没有', 75, 0.0063987714358843106),
 ('发现', 75, 0.0063987714358843106),
 ('加盟', 72, 0.006142820578448937),
 ('出来', 72, 0.006142820578448937),
 ('起来', 64, 0.005460284958621278),
 ('免费', 64, 0.005460284958621278),
 ('适合', 61, 0.005204334101185906),
 ('应该', 60, 0.005119017148707448),
 ('感觉', 59, 0.005033700196228991),
 ('可能', 57, 0.004863066291272076)]


In [50]:

# foreign words	
pprint(get_top([x for x in posdict['ws'] if len(x[0])>=4 and not x[0].startswith('http://')], num=25))


[('Nail', 179, 0.16950757575757575),
 ('Diamond', 131, 0.1240530303030303),
 ('FAME', 64, 0.06060606060606061),
 ('s.nail', 54, 0.05113636363636364),
 ('Salon', 23, 0.021780303030303032),
 ('nbsp', 21, 0.019886363636363636),
 ('nail', 14, 0.013257575757575758),
 ('Labo', 12, 0.011363636363636364),
 ('Project', 12, 0.011363636363636364),
 ('Helene', 12, 0.011363636363636364),
 ('O.P.I', 10, 0.00946969696969697),
 ('N.I.H', 7, 0.006628787878787879),
 ('Kesmoe', 7, 0.006628787878787879),
 ('uGalaxy', 7, 0.006628787878787879),
 ('coco', 6, 0.005681818181818182),
 ('blingbling', 6, 0.005681818181818182),
 ('S-NAIL', 5, 0.004734848484848485),
 ('Sebraner', 5, 0.004734848484848485),
 ('fafa', 5, 0.004734848484848485),
 ('miumiu', 4, 0.003787878787878788),
 ('STAR', 4, 0.003787878787878788),
 ('love', 4, 0.003787878787878788),
 ('Princess', 4, 0.003787878787878788),
 ('yeah', 4, 0.003787878787878788),
 ('BEnails', 4, 0.003787878787878788)]


In [51]:
# 命名实体
print(nedict.keys())


dict_keys(['O', 'B-Nh', 'E-Nh', 'S-Nh', 'S-Ns', 'B-Ns', 'E-Ns', 'I-Ns', 'B-Ni', 'E-Ni', 'I-Ni', 'S-Ni'])


In [52]:


# 人名
pprint(get_top(nedict['S-Nh'], num=25))


[('凤姐', 148, 0.12457912457912458),
 ('昕薇', 92, 0.07744107744107744),
 ('林尚美甲', 67, 0.0563973063973064),
 ('刘娟', 59, 0.049663299663299666),
 ('埃洛美甲', 55, 0.046296296296296294),
 ('芭比', 41, 0.034511784511784514),
 ('安娜', 33, 0.027777777777777776),
 ('周', 20, 0.016835016835016835),
 ('刘娟美', 17, 0.01430976430976431),
 ('阿叔', 13, 0.010942760942760943),
 ('萍萍', 11, 0.009259259259259259),
 ('张霞', 10, 0.008417508417508417),
 ('尚', 9, 0.007575757575757576),
 ('琳琳', 9, 0.007575757575757576),
 ('芳芳', 8, 0.006734006734006734),
 ('💅', 8, 0.006734006734006734),
 ('林尚', 8, 0.006734006734006734),
 ('罗玉凤', 7, 0.005892255892255892),
 ('陈', 7, 0.005892255892255892),
 ('陆惠', 6, 0.005050505050505051),
 ('荣氏', 6, 0.005050505050505051),
 ('艾维庭', 6, 0.005050505050505051),
 ('杨', 6, 0.005050505050505051),
 ('张', 6, 0.005050505050505051),
 ('华声', 5, 0.004208754208754209)]


In [53]:

# 机构名
pprint(get_top(nedict['S-Ni'], num=25))


[('美甲8', 1, 0.3333333333333333),
 ('新华社', 1, 0.3333333333333333),
 ('腾讯', 1, 0.3333333333333333)]


In [54]:

# 地名
pprint(get_top(nedict['S-Ns'], num=25))

[('美', 148, 0.10787172011661808),
 ('美国', 126, 0.09183673469387756),
 ('昆明', 124, 0.09037900874635568),
 ('瑞丽', 47, 0.034256559766763846),
 ('中国', 38, 0.027696793002915453),
 ('广州', 32, 0.023323615160349854),
 ('北京', 31, 0.02259475218658892),
 ('威海', 29, 0.021137026239067054),
 ('纽约', 23, 0.016763848396501458),
 ('海豚湾', 19, 0.013848396501457727),
 ('雁荡路', 18, 0.013119533527696793),
 ('香港', 18, 0.013119533527696793),
 ('上海', 18, 0.013119533527696793),
 ('三里屯', 17, 0.01239067055393586),
 ('梅陇镇', 16, 0.011661807580174927),
 ('延长路', 16, 0.011661807580174927),
 ('韩国', 16, 0.011661807580174927),
 ('日本', 15, 0.010932944606413994),
 ('玉指轩', 14, 0.01020408163265306),
 ('魏公村', 11, 0.008017492711370262),
 ('淮海中路', 11, 0.008017492711370262),
 ('朝阳区', 10, 0.007288629737609329),
 ('伊美店', 10, 0.007288629737609329),
 ('缤谷', 10, 0.007288629737609329),
 ('济宁', 9, 0.006559766763848397)]


In [ ]:
# 以上内容均为自动生成...